In [1]:
import cv2
import os
import glob
import numpy as np
import matplotlib.pyplot as plt
import math
import random

In [2]:
folder_path = "E:/UiS/DAT540/Group Project/images/"
# The folders Need to be such that -
    # 1 finger = one
    # 2 fingers = two
    # And so on
    # A zip file of the images folders is attached with the project submission.

In [3]:
# Loading files and putting them in lists
images = [cv2.imread(file) for file in glob.glob(folder_path + 'zero/*.jpg')]
images_real_results = [0 for file in glob.glob(folder_path + 'zero/*.jpg')]

images.extend([cv2.imread(file) for file in glob.glob(folder_path + 'one/*.jpg')])
images_real_results.extend([1 for file in glob.glob(folder_path + 'one/*.jpg')])

images.extend([cv2.imread(file) for file in glob.glob(folder_path + 'two/*.jpg')])
images_real_results.extend([2 for file in glob.glob(folder_path + 'two/*.jpg')])

images.extend([cv2.imread(file) for file in glob.glob(folder_path + 'three/*.jpg')])
images_real_results.extend([3 for file in glob.glob(folder_path + 'three/*.jpg')])

images.extend([cv2.imread(file) for file in glob.glob(folder_path + 'four/*.jpg')])
images_real_results.extend([4 for file in glob.glob(folder_path + 'four/*.jpg')])

images.extend([cv2.imread(file) for file in glob.glob(folder_path + 'five/*.jpg')])
images_real_results.extend([5 for file in glob.glob(folder_path + 'five/*.jpg')])

In [4]:
# First shuffle the data
# Since we have 2 lists and they reflect each others values,
# we had to combine them so that we can shuffle the values together,
# and finally separate them put them in their respective lists.
combined = list(zip(images, images_real_results))
random.shuffle(combined)
images[:], images_real_results[:] = zip(*combined)

# Separating the data into Training and Test data
# For this project we have separated them in a 66:34 ratio
split_index = int((66/100)*len(images))

training_images = images[:split_index]
training_image_results = images_real_results[:split_index]

test_images = images[split_index:]
test_image_results = images_real_results[split_index:]

print("No. of Training Images:",len(training_images))
print("No. of Test Images:", len(test_images))

No. of Training Images: 53
No. of Test Images: 28


In [5]:
def calc_fingers(count, arearatio, areacnt):
    if count == 0:
        if areacnt < 2000:
            fingers = 0
        else:
            if arearatio < 12:
                fingers = 0
            else:
                fingers = 1
    elif count == 1:
        fingers = 2
    elif count == 2:
        fingers = 3
    elif count == 3:
        fingers = 4
    elif count == 4:
        fingers = 5
    
    return fingers

def calculateFingers(image):  # -> finished bool, count: finger count
    drawing = np.zeros(image.shape, np.uint8)
    
    img = cv2.medianBlur(image, 5)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    blur = cv2.GaussianBlur(gray, (9, 11), 0)
    ret, thresh = cv2.threshold(blur, 127, 255, cv2.THRESH_BINARY)
    
    thresh = (255 - thresh)
    
    drawing = thresh
    
    _, contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
    cnt = max(contours, key = lambda x: cv2.contourArea(x))
    
    length = len(contours)
    
    hull1 = cv2.convexHull(cnt)
        #print('here')
    cv2.drawContours(drawing, [cnt], 0, (0, 255, 0), 2)
    cv2.drawContours(drawing, [hull1], 0, (0, 0, 255), 2)
        
    hull = cv2.convexHull(cnt, returnPoints=False)
    areacnt = cv2.contourArea(cnt)
    areahull = cv2.contourArea(hull1)
    arearatio=((areahull-areacnt)/areacnt)*100
#     print(arearatio)
#     print(areacnt)
#     print(areahull)
    if len(hull) > 3:
        defects = cv2.convexityDefects(cnt, hull)

        if type(defects) != type(None):  # avoid crashing.   (BUG not found)
            count = 0
            for i in range(defects.shape[0]):  # calculate the angle
                s, e, f, d = defects[i][0]
                start = tuple(cnt[s][0])
                end = tuple(cnt[e][0])
                far = tuple(cnt[f][0])



                a = math.sqrt((end[0] - start[0]) ** 2 + (end[1] - start[1]) ** 2)
                b = math.sqrt((far[0] - start[0]) ** 2 + (far[1] - start[1]) ** 2)
                c = math.sqrt((end[0] - far[0]) ** 2 + (end[1] - far[1]) ** 2)

                angle = math.acos((b ** 2 + c ** 2 - a ** 2) / (2 * b * c))  # cosine theorem
#                     print(d)
                if angle <= math.pi / 3 and d > 1000:  # angle less than 90 degree, treat as fingers
                    count += 1
                    cv2.circle(drawing, far, 4, [255, 255, 255], -1)
                
            #plt.imshow(drawing)
            #print(count)
            return calc_fingers(count,arearatio, areacnt)
    return 0

In [7]:
n_defects = []
#cv_img = []

for i in range(len(training_images)):
    #plt.imshow(img)
    #print(i)
    n_defects.append(calculateFingers(training_images[i]))

# Output
fmt = '{:<8}{:<20}{}'
print(fmt.format('', 'Defects', 'Number of Fingers'))
for i, (defect, no_of_fingers) in enumerate(zip(n_defects, training_image_results)):
    print(fmt.format(i+1, defect, no_of_fingers))

        Defects             Number of Fingers
1       2                   2
2       2                   2
3       5                   4
4       3                   3
5       2                   0
6       3                   2
7       1                   2
8       2                   1
9       3                   4
10      2                   2
11      2                   0
12      2                   1
13      2                   4
14      4                   3
15      1                   1
16      4                   4
17      5                   5
18      0                   2
19      2                   0
20      1                   0
21      3                   5
22      2                   3
23      3                   3
24      2                   2
25      5                   4
26      4                   4
27      1                   1
28      2                   2
29      2                   3
30      2                   2
31      4                   5
32      0               

In [8]:
# Importing KNN from Scikit-Learn
from sklearn.neighbors import KNeighborsClassifier

features = list(zip(n_defects))
    
model = KNeighborsClassifier(n_neighbors = 3)
model.fit(features, training_image_results)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=3, p=2,
           weights='uniform')

In [9]:
# Function for predicting with KNN
def knn_prediction(predict_for):
    prediction = model.predict([[predict_for]]) # Prediction with 'n' defects
    return prediction[0]

In [11]:
# The Accuracy of the model

#fmt = '{:<8}{:<20}{:<20}{}'
#print(fmt.format('', 'Defects', 'Prediction', 'Real Number of Fingers'))
correct_count = 0

for i in range(len(test_images)):
    defect = calculateFingers(test_images[i])

    prediction = knn_prediction(predict_for = defect)

    #print(fmt.format(i+1, defect, prediction, test_image_results[i]))

    if(prediction == test_image_results[i]):
        correct_count = correct_count + 1
        
print("Total test images:",len(test_images))
print("Number of correct predictions:",correct_count)
print("\nAccuracy:", 100*(correct_count/len(test_images)))

Total test images: 28
Number of correct predictions: 9

Accuracy: 32.142857142857146


In [12]:
n_defects = []

for img in images:
    n_defects.append(calculateFingers(img))
    
features = list(zip(n_defects))
    
model = KNeighborsClassifier(n_neighbors = 3)
model.fit(features, images_real_results)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=3, p=2,
           weights='uniform')

In [ ]:
captured_image = None

cam = cv2.VideoCapture(0)

cv2.namedWindow("hand gesture")

img_counter = 0

while cam.isOpened():
    ret, frame = cam.read()
    
    cv2.rectangle(frame,(70,70),(300,300),(100,200,50),0)
    crop_image = frame[90:300, 90:300]
    
    cv2.imshow("hand gesture", frame)
    
    if not ret:
        break
    k = cv2.waitKey(1)

    if k%256 == 27:
        # ESC pressed
        break
    elif k%256 == 32: # SPACE pressed
        captured_image = crop_image
        break
        
cam.release()

cv2.destroyAllWindows()

# Predict the Number of Fingers in the Captured Image.
defect = calculateFingers(captured_image)

prediction = knn_prediction(predict_for = defect)

fmt = '{:<20}{}'
print(fmt.format('Defects Found', 'Prediction'))
print(fmt.format(defect, prediction))
plt.imshow(captured_image)